In [ ]:
import os
from dotenv import load_dotenv
from ibm_cloud_sdk_core import IAMTokenManager
import warnings
warnings.filterwarnings('ignore')

load_dotenv('config.env')

# Connection variables
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None) 
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
access_token = IAMTokenManager(
    apikey = api_key,
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

In [1]:
from sentence_transformers import SentenceTransformer
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

import os 

host = os.getenv("MILVUS_HOST", None)
port = os.getenv("MILVUS_PORT", None)
password = os.getenv("LH_PW", None)
user = os.getenv("LH_USER", None)
server_pem_path = os.getenv("LH_CERT", None)

connections.connect(alias = 'default',
                   host = host,
                   port = port,
                   user = user,
                   password = password,
                   server_pem_path = server_pem_path,
                   server_name = host,
                   secure = True)

# Load collection

basic_collection = Collection("wiki_articles")      
basic_collection.load()

# Query function
def query_milvus(query, num_results=5):
    
    # Vectorize query
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # 384 dim
    query_embeddings = model.encode([query])

    # Search
    search_params = {
        "metric_type": "L2", 
        "params": {"nprobe": 5}
    }
    results = basic_collection.search(
        data=query_embeddings, 
        anns_field="vector", 
        param=search_params,
        limit=num_results,
        expr=None, 
        output_fields=['article_text'],
    )
    return results

ConnectionConfigException: <ConnectionConfigException: (code=1, message=Type of 'host' must be str.)>

In [ ]:
## Consider how climate change may relate to other industries and processes related to your business

question_text = "What can my company do to help fight climate change?"

In [ ]:
# Query Milvus 

def get_chunks(query):
    num_results = 3
    results = query_milvus(query, num_results)

    relevant_chunks = []
    for i in range(num_results):    
        #print(f"id: {results[0].ids[i]}")
        #print(f"distance: {results[0].distances[i]}")
        text = results[0][i].entity.get('article_text')
        relevant_chunks.append(text)
    
    #print(relevant_chunks)
    return relevant_chunks

In [ ]:
def make_prompt(context, question_text):
    context = "\n\n".join(context)
    return (f"{context}\n\nPlease answer a question using this text. "
          + f"If the question is unanswerable, say \"unanswerable\"."
          + f"\n\nQuestion: {question_text}")


In [20]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# Model Parameters
params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 500,
        GenParams.TEMPERATURE: 0,
}
model = Model(
#        model_id='meta-llama/llama-2-70b-chat', 
        model_id='mistralai/mixtral-8x7b-instruct-v01',
        params=params, credentials=creds, 
        project_id=project_id
)

# Prompt LLM
def ask_llm(prompt):
        response = model.generate_text(prompt)
        #print(f"Question: {question_text}{response}")
        return response

In [21]:
from ipywidgets import widgets

text_input = widgets.Textarea(value=question_text, disabled=False)
result_text = widgets.Textarea(value='', disabled=True)
prompt_text = widgets.Textarea(value='', disabled=True)

def on_click(b):
#    display('You clicked the button!' + text_input.value + " radio=" + radio.value)
    if (radio.value == "without RAG"):
        result_text.value = "asking AI (without RAG) ..."
        prompt = text_input.value
    else:
        result_text.value = "asking AI (with RAG) ..."
        chunks = get_chunks(text_input.value)
        prompt = make_prompt(chunks, text_input.value)
    prompt_text.value = prompt
    result_text.value = ask_llm(prompt)

button = widgets.Button(description='Ask AI');
button.on_click(on_click)

radio=widgets.RadioButtons(options=['without RAG', 'with RAG'], description='Please choose')

radio_box  = widgets.HBox([radio])
input_box  = widgets.Box([widgets.Label('Your question!'), text_input, button])
result_box = widgets.Box([result_text])
prompt_box = widgets.Box([prompt_text])

box = widgets.VBox(children=[radio_box, input_box , result_box, prompt_box])

result_text.layout.width = '100%'
result_text.layout.height = '200px'
prompt_text.layout.width = '100%'

display(box)
